In [1]:
from inspect import Signature, Parameter

# Создаем сигнатуру для func(x, y=42, *, z=None)
parms = [
    Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
    Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
    Parameter('z', Parameter.KEYWORD_ONLY, default=None)
]

sig = Signature(parms)
print(sig)
print('-----------------')

def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name,value)
        
func(1, 2, z=3)
print('-----------------')
func(1, z=3)
print('-----------------')
func(y=2, x=1)

(x, y=42, *, z=None)
-----------------
x 1
y 2
z 3
-----------------
x 1
z 3
-----------------
x 1
y 2


In [2]:
import inspect
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()

    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)


# Пример использования
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')

class Point(Structure):
    __signature__ = make_sig('x', 'y')
    

print(inspect.signature(Stock))

(name, shares, price)


In [3]:
import inspect
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names]
    return Signature(parms)


class StructureMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = make_sig(*clsdict.get('_fields',[]))
        return super().__new__(cls, clsname, bases, clsdict)
    
class Structure(metaclass=StructureMeta):
    _fields = []
    
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)
            

# Пример
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x', 'y']
    
print(inspect.signature(Stock))
print(inspect.signature(Point))

(name, shares, price)
(x, y)


In [2]:
from inspect import signature
import logging

class MatchSignaturesMeta(type):
    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)
        sup = super(self, self)
        for name, value in clsdict.items():
            if name.startswith('_') or not callable(value):
                continue
            # Получить предыдущее определение
            # (если оно есть) и сравнение сигнатур
            prev_dfn = getattr(sup, name, None)
            if prev_dfn:
                prev_sig = signature(prev_dfn)
                val_sig = signature(value)
                if prev_sig != val_sig:
                    logging.warning('Signature mismatch in %s. %s != %s', value.__qualname__, prev_sig, val_sig)
                    

# Пример
class Root(metaclass=MatchSignaturesMeta):
    pass

class A(Root):
    def foo(self, x, y):
        pass
    
    def spam(self, x, *, z):
        pass

# Класс с переопределенными методами,
# но со слегка различными сигнатурами
class B(A):
    def foo(self, a, b):
        pass
    def spam(self,x,z):
        pass
    

b = B()